In [1]:
import ee
import geemap
import sys
sys.path.insert(0, '../RadGEEToolbox')
import LandsatCollection
import Sentinel2Collection
import GetPalette
import VisParams

In [2]:
ee.Initialize()

#### Many attributes, property attributes, methods, and static methods from the RadGEEToolbox package allow for calculating multispectral indices such as NDWI (water), NDVI (vegetation), NDTI (water turbidity), relative chlorophyll-a concentrations, and other indices for mapping evaporites (established by the author of this package - see Radwin & Bowen, 2021). Additionally, Land Surface Temperature (LST) can be easily calculcated for Landsat imagery.

#### In addition to multispectral indices, there are easy ways to temporally reduce image collections to mean, median, min, or max images

### Let's start with the attributes for multispectral processing - below is a list of the useful attributes and property attributes for the LandsatCollection class
    
    Attributes:
        collection: Returns an ee.ImageCollection object from any LandsatCollection image collection object
        

    Property attributes:
        dates_list (returns: Server-Side List): Unreadable Earth Engine list of image dates (server-side)
        
        dates (returns: Client-Side List): Readable pythonic list of image dates (client-side)
        
        masked_clouds_collection (returns: LandsatCollection image collection): Returns collection with clouds masked (transparent) for each image

        masked_water_collection (returns: LandsatCollection image collection): Returns collection with water pixels masked (transparent) for each image

        masked_to_water_collection (returns: LandsatCollection image collection): Returns collection with pixels masked to water (transparent) for each image (masks land and cloud pixels)

        max (returns: ee.Image): Returns a temporally reduced max image (calculates max at each pixel)
        
        median (returns: ee.Image): Returns a temporally reduced median image (calculates median at each pixel)
        
        mean (returns: ee.Image): Returns a temporally reduced mean image (calculates mean at each pixel)
        
        min (returns: ee.Image): Returns a temporally reduced min image (calculates min at each pixel)
        
        MosaicByDate (returns: LandsatCollection image collection): Mosaics image collection where images with the same date are mosaiced into the same image. Calculates total cloud percentage for subsequent filtering of cloudy mosaics.
        
        gypsum (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband gypsum index rasters
        
        halite (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband halite index rasters
        
        LST (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband land-surface-temperature rasters (Celcius)
        
        ndwi (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband NDWI (water) rasters
        
        ndvi (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband NDVI (vegetation) rasters

        turbidity (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband NDTI (turbidity) rasters

        chlorophyll (returns: ee.ImageCollection): Returns LandsatCollection image collection of singleband KIVU (relative chlorophyll-a) rasters

In [3]:
# First we need an image collection
col = LandsatCollection.LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', tile_row=32, tile_path=38)

#Then it is as easy as calling the desired attribute for multispectral indices
NDWI = col.ndwi
NDVI = col.ndvi
NDTI = col.turbidity
halite = col.halite
gypsum = col.gypsum
chlorophyll = col.chlorophyll
surface_temperature = col.LST

print(NDWI.image_grab(-1).getInfo())
print(surface_temperature.image_grab(-1).select('LST').getInfo())

# Temporal reductions of image collections
mean_img = col.mean
min_img = col.min
median_img = col.median
max_img = col.max

#Temporal reductions can be applied to multispectral indices as well
mean_chlorophyll = chlorophyll.mean


{'type': 'Image', 'bands': [{'id': 'ndwi', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [7801, 7911], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 299985, 0, -30, 4583415]}], 'properties': {'DATUM': 'WGS84', 'REFLECTANCE_ADD_BAND_3': -0.2, 'REFLECTANCE_ADD_BAND_4': -0.2, 'REFLECTANCE_ADD_BAND_5': -0.2, 'REFLECTANCE_ADD_BAND_6': -0.2, 'REFLECTANCE_ADD_BAND_7': -0.2, 'REFLECTIVE_SAMPLES': 7801, 'GROUND_CONTROL_POINTS_VERSION': 5, 'SUN_AZIMUTH': 127.82804601, 'UTM_ZONE': 12, 'DATE_ACQUIRED': '2023-06-29', 'ELLIPSOID': 'WGS84', 'DATA_SOURCE_PRESSURE': 'Calculated', 'LANDSAT_PRODUCT_ID': 'LC09_L2SP_038032_20230629_20230703_02_T1', 'STATION_ID': 'LGN', 'TEMPERATURE_ADD_BAND_ST_B10': 149, 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT', 'REFLECTANCE_MULT_BAND_7': 2.75e-05, 'REFLECTANCE_MULT_BAND_6': 2.75e-05, 'L1_PROCESSING_LEVEL': 'L1TP', 'PROCESSING_SOFTWARE_VERSION': 'LPGS_16.3.0', 'L1_DATE_PRODUCT_GENERATED': '2023-06-29T22:49:40Z', 'ORIENTATION':

#### Note that by using attributes you use the already defined thresholds for masking the images, which is typically -1 for normalized indices and results in no pixels being masked. If you want to mask each image to pixels above a threshold, you can use method functions and specify the desired threshold.
    Methods:
        get_filtered_collection(self)

        get_boundary_filtered_collection(self)
        
        ndwi_collection(self, threshold, ng_threshold=None)
        
        ndvi_collection(self, threshold, ng_threshold=None)
        
        halite_collection(self, threshold, ng_threshold=None)
        
        gypsum_collection(self, threshold, ng_threshold=None)

        turbidity_collection(self, threshold, ng_threshold=None)

        chlorophyll_collection(self, threshold, ng_threshold=None)

        masked_water_collection_NDWI(self, threshold)

        masked_to_water_collection_NDWI(self, threshold)
        
        surface_temperature_collection(self)
        
        mask_to_polygon(self, polygon)

        mask_out_polygon(self, polygon)
        
        mask_halite(self, threshold, ng_threshold=None)
        
        mask_halite_and_gypsum(self, halite_threshold, gypsum_threshold, halite_ng_threshold=None, gypsum_ng_threshold=None)
        
        image_grab(self, img_selector)
        
        custom_image_grab(self, img_col, img_selector)
        
        image_pick(self, img_date)
        
        CollectionStitch(self, img_col2)

In [4]:
#Using methods to calculate multispectral indices

NDWI = col.ndwi_collection(threshold=0)
NDVI = col.ndvi_collection(threshold=0.3)
turbidity = col.turbidity_collection(threshold=0)

#If you will be using Landsat 5 and Landsat 8 or 9 imagery together in a collection, you can specify different thresholds 
# for TM vs OLI sensors where ng_threshold means "next generation" threshold for newer sensors
NDWI = col.ndwi_collection(threshold=0, ng_threshold=0.2)

#### Similar goes for any static methods, you can use these to process multispectral indices for Earth Engine image objects
    Static Methods:
        image_dater(image)
        
        landsat5bandrename(img)
        
        landsat_ndwi_fn(image, threshold, ng_threshold=None)
        
        landsat_ndvi_fn(image, threshold, ng_threshold=None)
        
        landsat_halite_fn(image, threshold, ng_threshold=None)
        
        landsat_gypsum_fn(image, threshold, ng_threshold=None)

        landsat_ndti_fn(image, threshold, ng_threshold=None)

        landsat_kivu_chla_fn(image, threshold, ng_threshold=None)
        
        MaskWaterLandsat(image)

        MaskToWaterLandsat(image)

        MaskWaterLandsatByNDWI(image, threshold, ng_threshold=None)

        MaskToWaterLandsatByNDWI(image, threshold, ng_threshold=None)
        
        halite_mask(image, threshold, ng_threshold=None)
        
        gypsum_and_halite_mask(image, halite_threshold, gypsum_threshold, halite_ng_threshold=None, gypsum_ng_threshold=None)
        
        maskL8clouds(image)
        
        temperature_bands(img)
        
        landsat_LST(image)
        
        PixelAreaSum(image, band_name, geometry, threshold=-1, scale=30, maxPixels=1e12)
        
        dNDWIPixelAreaSum(image, geometry, band_name='ndwi', scale=30, maxPixels=1e12)

In [5]:
#Example using static method function to calculate NDWI for all images in an ee.ImageCollection object
#Note the need for lambda to map functions with arguments other than the input image
NDWI = col.collection.map(lambda image: LandsatCollection.LandsatCollection.landsat_ndwi_fn(image, threshold=0))
print(NDWI.getInfo())



{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'ndwi', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 302085, 0, -30, 4583415]}], 'properties': {'DATUM': 'WGS84', 'REFLECTANCE_ADD_BAND_3': -0.2, 'REFLECTANCE_ADD_BAND_4': -0.2, 'REFLECTANCE_ADD_BAND_5': -0.2, 'REFLECTANCE_ADD_BAND_6': -0.2, 'REFLECTANCE_ADD_BAND_7': -0.2, 'REFLECTIVE_SAMPLES': 7801, 'GROUND_CONTROL_POINTS_VERSION': 5, 'SUN_AZIMUTH': 130.95887584, 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION': 'TIRS', 'UTM_ZONE': 12, 'DATE_ACQUIRED': '2023-06-05', 'ELLIPSOID': 'WGS84', 'DATA_SOURCE_PRESSURE': 'Calculated', 'LANDSAT_PRODUCT_ID': 'LC08_L2SP_038032_20230605_20230613_02_T1', 'STATION_ID': 'LGN', 'TEMPERATURE_ADD_BAND_ST_B10': 149, 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT', 'REFLECTANCE_MULT_BAND_7': 2.75e-05, 'REFLECTANCE_MULT_BAND_6': 2.75e-05, 'L1_PROCESSING_LEVEL': 'L1TP', 'PROCESS